In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from peft import PeftModel
import numpy as np

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [2]:
import os
import sys

# Get the absolute path of the current script's directory
current_dir = os.path.dirname(os.path.abspath("gemini2.5.ipynb"))

# Get the absolute path of the parent directory (project_folder)
parent_dir = os.path.dirname(current_dir)

# Add the parent directory to the Python path
sys.path.append(parent_dir)

# Now you can import from GetXY.py
from GetXY import expressions_not_in_x, outsideExpr, longer_Exps, y_test, out_y_test, long_y_test, x_string, y

x_test = expressions_not_in_x
out_x_test = outsideExpr
long_x_test = longer_Exps
# ... rest of your code
print("Successfully imported variables!")

3 + 2 - -4
2543
9.0

Expressions not in x:
3 - 4 + -1
True
1457
-2.0
15
-4.0
[-5.   1.   1.   0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5
  0.5]
Successfully imported variables!


In [3]:
tokenizer = AutoTokenizer.from_pretrained("./output1")


In [4]:
# The base model ID, as indicated by the error message
base_model_id = "google/gemma-3-270m-it"

# Your fine-tuned adapter
adapter_id = "AntonBOOM/output1"
#commit_hash = "eebf1c3f9adbfe9cdcc7067cbaf5d20e5828610d"

# 1. Load the base model
model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True
)

# 2. Load the Peft model and merge it with the base model
model = PeftModel.from_pretrained(model, adapter_id, revision = "main")
model = model.merge_and_unload()

# The 'model' variable now holds your fully fine-tuned model

adapter_config.json:   0%|          | 0.00/934 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/30.4M [00:00<?, ?B/s]

In [5]:
inputs1 = []

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    print("added padding")
for i in x_test:
    messages = [
        {"role": "user", "content": i}
    ]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs1.append(tokenizer(prompt, return_tensors="pt").to(model.device))
print(inputs1[0])

inputs2 = []

for i in x_string:
    messages = [
        {"role": "user", "content": i}
    ]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs2.append(tokenizer(prompt, return_tensors="pt").to(model.device))

{'input_ids': tensor([[     2,      2,    105,   2364,    107, 236800,    753, 236743, 236812,
            900,    753, 236770,    106,    107,    105,   4368,    107]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}


In [6]:
def decodePreds(output):
    responses = []
    turn_separator = "<start_of_turn>model\n"
    decoded_preds = tokenizer.batch_decode(output, skip_special_tokens=False)
    for i in decoded_preds:
        pred_response = i.split(turn_separator, 1)[1]
        clean_pred = pred_response.replace("<end_of_turn>", "") \
                                  .replace("<pad>", "") \
                                  .replace("<start_of_turn>", "") \
                                  .replace("model", "") \
                                  .replace("\n", "") \
                                  .strip()
    
    #print(clean_pred)
    responses.append(clean_pred)
    
    return responses

In [7]:
from transformers import logging
from transformers import pipeline
#x_test = x_string
#y_test = y

x_test = x_test[:100]
y_test_float = [float(i) for i in y_test]

generator = pipeline("text-generation", model="output", device="cuda")

logging.set_verbosity_info()
responses3 = []
counter = 0
for i in x_test:
    question = i
    output = generator([{"role": "user", "content": question}], max_new_tokens=128, return_full_text=False)[0]
    responses3.append(output["generated_text"])
    counter += 1
    print(counter/len(x_test))





Device set to use cuda


0.01
0.02
0.03
0.04
0.05
0.06
0.07
0.08
0.09


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


0.1
0.11
0.12
0.13
0.14
0.15
0.16
0.17
0.18
0.19
0.2
0.21
0.22
0.23
0.24
0.25
0.26
0.27
0.28
0.29
0.3
0.31
0.32
0.33
0.34
0.35
0.36
0.37
0.38
0.39
0.4
0.41
0.42
0.43
0.44
0.45
0.46
0.47
0.48
0.49
0.5
0.51
0.52
0.53
0.54
0.55
0.56
0.57
0.58
0.59
0.6
0.61
0.62
0.63
0.64
0.65
0.66
0.67
0.68
0.69
0.7
0.71
0.72
0.73
0.74
0.75
0.76
0.77
0.78
0.79
0.8
0.81
0.82
0.83
0.84
0.85
0.86
0.87
0.88
0.89
0.9
0.91
0.92
0.93
0.94
0.95
0.96
0.97
0.98
0.99
1.0


In [8]:
y_test = y_test[:100]
float_preds3 = [float(i) for i in responses3]
diffs = []
for i in range(len(y_test)):
    diff = y_test_float[i] - float_preds3[i]
    diffs.append(abs(diff))
print(np.mean(diffs))

count = 0
responses3 = [str(i) for i in responses3]
y_test = [str(i) for i in y_test]

for i in range(len(y_test)):
    if y_test[i] == responses3[i]:
        count += 1
    else: print(y_test[i], responses3[i])
print(count/len(y_test))

0.66
-4.0 -2.0
-1.0 0.0
0.0 2.0
-11.0 -5.0
-10.0 -8.0
4.0 6.0
-1.0 1.0
-1.0 1.0
-5.0 -3.0
-7.0 -5.0
-3.0 -3
-3.0 -1.0
-6.0 -4.0
2.0 4.0
2.0 0.0
4.0 6.0
7.0 5.0
-1.0 1.0
-6.0 -4.0
-1.0 1.0
-1.0 0.0
1.0 5.0
2.0 4.0
0.0 2.0
3.0 5.0
-2.0 0.0
3.0 5.0
2.0 4.0
-8.0 -6.0
-3.0 -5.0
-3.0 -1
-2.0 0.0
0.68


In [9]:
responses1 = []
for input in inputs1:
    output = model.generate(
        **input,
        max_new_tokens=20, 
        do_sample=False,
        temperature=None, # Explicitly unset
        top_p=None,       # Explicitly unset
        top_k=None        # Explicitly unset
    )

    responses1.append(decodePreds(output))

Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:1 for o

KeyboardInterrupt: 

In [ ]:
responses2 = []
for input in inputs2:
    output = model.generate(
        **input,
        max_new_tokens=20, 
        do_sample=False,
        temperature=None, # Explicitly unset
        top_p=None,       # Explicitly unset
        top_k=None        # Explicitly unset
    )
    responses2.append(decodePreds(output))

In [ ]:
print(responses1[:5])
print(responses2[:5])

In [ ]:
float_preds1 = [float(i[0]) for i in responses1]
string_preds1 = [str(i[0]) for i in responses1]
float_preds2 = [float(i[0]) for i in responses2]
string_preds2 = [str(i[0]) for i in responses2]
y_test_float = [float(i) for i in y_test]
y_test_string = [str(i) for i in y_test]
y_float = [float(i) for i in y]
y_string = [str(i) for i in y]

In [ ]:
diffs = []
for i in range(len(y_test)):
    diff = y_test_float[i] - float_preds1[i]
    diffs.append(abs(diff))
print(np.mean(diffs))

In [ ]:
diffs2 = []
for i in range(len(y_float)):
    diff = y_float[i] - float_preds2[i]
    diffs2.append(abs(diff))
print(np.mean(diffs2))

In [ ]:
count = 0
for i in range(len(y_test_string)):
    if y_test_string[i] == string_preds1[i]:
        count += 1
print(count/len(y_test_string))

In [ ]:
count = 0
for i in range(len(y_string)):
    if y_string[i] == string_preds2[i]:
        count += 1
print(count/len(y_string))

In [ ]:
from transformers import pipeline

question = "1 + 2 + 3"
generator = pipeline("text-generation", model="AntonBOOM/output", device="cuda")
output = generator([{"role": "user", "content": question}], max_new_tokens=128, return_full_text=False)[0]
print(output["generated_text"])


In [ ]:
diffs = []
for i in range(len(y_test)):
    diff = y_test_float[i] - float_preds3[i]
    diffs.append(abs(diff))
print(np.mean(diffs))